In [1]:
import pandas as pd

from Reprezentacja_sumacyjna import CreateTable

In [2]:
from typing import List, Tuple

In [3]:
df = CreateTable("0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15")

0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 


In [5]:
df_groupwane = pd.DataFrame(
    columns=["Obiekt grupowany", "Elemtny(łączone)", "Element", "Użyty"],
    data=[[0, 0, '0000', False],
            [1, 2, '0010', False],
            [1, 8, '1000', False],
            [2, 3, '0011', False],
            [2, 5, '0101', False],
            [2, 6, '0110', False],
            [2, 9, '1001', False],
            [2, 10, '1010', False],
            [2, 12, '1100', False],
            [3, 7, '0111', False],
            [3, 11, '1011', False],
            [3, 13, '1101', False],
            [3, 14, '1110', False],
            [4, 15, '1111', False]])
df_groupwane

,Obiekt grupowany,Elemtny(łączone),Element,Użyty
0,0,0,0000,False
1,1,2,0010,False
2,1,8,1000,False
3,2,3,0011,False
4,2,5,0101,False
5,2,6,0110,False
6,2,9,1001,False
7,2,10,1010,False
8,2,12,1100,False
9,3,7,0111,False


In [6]:
df_glowna_wynik = pd.DataFrame(
    columns=["Obiekt grupowany", "Elemtny(łączone)", "Element", "Użyty"],
    data=[[0, 0, '0000', True],
          [1, 2, '0010', True],
          [1, 8, '1000', True],
          [2, 5, '0101', False],
          [2, 6, '0110', False],
          [2, 9, '1001', False],
          [2, 10, '1010', False],
          [2, 12, '1100', False],
          [3, 7, '0111', False],
          [3, 11, '1011', False],
          [3, 13, '1101', False],
          [3, 14, '1110', False],
          [4, 15, '1111', False],
          ]
)
df_glowna_wynik

,Obiekt grupowany,Elemtny(łączone),Element,Użyty
0,0,0,0000,True
1,1,2,0010,True
2,1,8,1000,True
3,2,5,0101,False
4,2,6,0110,False
5,2,9,1001,False
6,2,10,1010,False
7,2,12,1100,False
8,3,7,0111,False
9,3,11,1011,False


In [7]:
df_wynikowa1 = pd.DataFrame(
    columns=["Obiekt grupowany", "Elemtny(łączone)", "Element", "Użyty"],
    data=[[(0, 1), (0, 2), '00-0', False],
          [(0, 1), (0, 8), '-000', False]]
)
df_wynikowa1

,Obiekt grupowany,Elemtny(łączone),Element,Użyty
0,"(0, 1)","(0, 2)",00-0,False
1,"(0, 1)","(0, 8)",-000,False


In [8]:
def matched(num1: str, num2: str) -> bool:
    return sum(n1 != n2 for n1, n2 in zip(num1, num2)) == 1

In [9]:
assert matched('0000', '000x') == True

In [10]:
def correct_pair(num1: str, num2: str) -> str:
    return ''.join(n1 if n1 == n2 else '-' for n1, n2 in zip(num1, num2))

In [11]:
def correct_tuple(tuple_elements: Tuple) -> Tuple:
    tup_element = tuple_elements
    # print(f"tupl_elem, {tup_element}")
    while any(isinstance(el, tuple) for el in tup_element):
        tup_element = sum(tup_element, ())
    tup_element = tuple(set(tup_element))
    return tup_element

In [12]:
def elementy_laczone(df_input: pd.DataFrame) -> list:
    return list(df_input["Elemtny(łączone)"])

In [13]:
def remove_element(lista_a, element):
    if element in lista_a:
        # print(f"Delete {element}, actual list: {lista_a}")
        lista_a.remove(element)

In [14]:
def compare_neighbours(df_input: pd.DataFrame, main: int, next_: int, el_lacz: list=[])\
        -> Tuple[pd.DataFrame, list]:
    # print(f"El_laczone = {el_lacz}")
    df_new = pd.DataFrame(columns=["Obiekt grupowany", "Elemtny(łączone)", "Element", "Użyty"])
    df_main = df_input.copy()
    # print each comparsion
    main_rows = df_main[df_main["Obiekt grupowany"] == main]
    next_rows = df_main[df_main["Obiekt grupowany"] == next_]
    # print(f"next->{next_rows}")
    komu_true = set()

    for idx_main, row_main in main_rows.iterrows():
        for idx_, row_ in next_rows.iterrows():
            check1 = row_main['Element']
            check2 = row_['Element']
            if matched(check1, check2):
                # print(f"Matched! -{check1}- with {check2}")
                # print(f"nr -{row_main['Elemtny(łączone)']}- with {row_['Elemtny(łączone)']}")
                remove_element(el_lacz, row_main["Elemtny(łączone)"])
                remove_element(el_lacz, row_["Elemtny(łączone)"])
                if df_new.empty:
                    x = 0
                    df_new["Użyty"] = df_new["Użyty"].astype(bool)
                else:
                    x = df_new.index.max() + 1
                grupowy_tuple = correct_tuple(tuple([main]) + tuple([next_]))
                laczony_tuple = correct_tuple(tuple([row_main["Elemtny(łączone)"]]) + tuple([row_["Elemtny(łączone)"]]))
                df_new.loc[x] = [grupowy_tuple,
                                 laczony_tuple,
                                 correct_pair(check1, check2),
                                 False]
                ### do glownej tablicy trzeba zaznaczyc, ze dla row i dla row_ <- jest True
                komu_true.update([
                    row_main["Elemtny(łączone)"],
                    row_["Elemtny(łączone)"]
                ])
    # for item in komu_true:
    #     # print(f"item = {item}")
    #     idx_df = df_groupwane[df_groupwane["Elemtny(łączone)"] == item].index[0]
    #     df_main.at[idx_df, "Użyty"] = True
                    #df_main[df_main["Elemtny(łączone)"] == item]["Użyty"] = True
        # print(f"Te elementy na True: {komu_true}")
    return df_new, el_lacz

In [15]:
list(df_groupwane["Elemtny(łączone)"])

[0, 2, 8, 3, 5, 6, 9, 10, 12, 7, 11, 13, 14, 15]

In [41]:
def create_group(df_input: pd.DataFrame, lista_lacz=[]) -> pd.DataFrame:
    df_main = df_input.copy()
    df_out = pd.DataFrame()
    lista_lacz = lista_lacz + elementy_laczone(df_input)
    unique_values = list(df_input['Obiekt grupowany'].unique())
    # print(f"Unique elements len = {len(unique_values)}")
    # print(f"Unique elements = {unique_values}")
    # print(f"Uprise group len = ->{len(unique_values) - 1}<-")
    for idx, el in enumerate(unique_values):
        if el == unique_values[-1]: continue
        df_tmp, lista_lacz = compare_neighbours(df_main, unique_values[idx], unique_values[idx+1], lista_lacz)
        df_out = pd.concat([df_out, df_tmp])
    df_out = df_out.drop_duplicates(subset='Element')
    return df_out.reset_index(drop=True), lista_lacz

In [42]:
def all_groups(df_input: pd.DataFrame):
    df_all: List[pd.DataFrame] = []
    implicants: list = []
    df_out, list_out = create_group(df_input)
    while not df_out.empty:
        df_out, list_out = create_group(df_out, list_out)
    print(f"List out: {list_out}")
    return list_out

In [45]:
all_groups(df_groupwane)

List out: [(0, 8, 2, 10), (13, 15, 5, 7), (2, 3, 6, 7, 10, 11, 14, 15), (8, 9, 10, 11, 12, 13, 14, 15)]


[(0, 8, 2, 10),
 (13, 15, 5, 7),
 (2, 3, 6, 7, 10, 11, 14, 15),
 (8, 9, 10, 11, 12, 13, 14, 15)]

In [103]:
def create_tablica_pokryc(postac_sumacyjna: List[int], implikanty_proste: dict):
    print(f"Postac sumacyjna: {postac_sumacyjna}")
    print("Słownik")
    for key, item in implikanty_proste.items():
        print(f"key[{key}] = {item}")

In [101]:
from typing import Dict

In [104]:
postac_sumacyjna: List[int] = [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
implikanty_proste: Dict[str, tuple] = {'-0-0': (0, 8, 2, 10),
                                       '-1-1': (5, 7, 13, 15),
                                       '--1-': (2, 3, 6, 7, 10, 11, 14, 15),
                                       '1---': (8, 9, 11, 12, 13, 14, 15)}
create_tablica_pokryc(postac_sumacyjna, implikanty_proste)

Postac sumacyjna: [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Słownik
key[-0-0] = (0, 8, 2, 10)
key[-1-1] = (5, 7, 13, 15)
key[--1-] = (2, 3, 6, 7, 10, 11, 14, 15)
key[1---] = (8, 9, 11, 12, 13, 14, 15)


In [95]:
df_tablica_pokryc = pd.DataFrame(
    index = [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    columns=['-0-0', '-1-1', '--1-', '1---'],
    data=[['+', '-', '-', '-'],
          ['+', '-', '+', '-'],
          ['-', '-', '+', '-'],
          ['-', '+', '-', '-'],
          ['-', '-', '+', '-'],
          ['-', '+', '+', '-'],
          ['+', '-', '-', '+'],
          ['-', '-', '-', '+'],
          ['+', '-', '+', '+'],
          ['-', '-', '+', '+'],
          ['-', '-', '-', '+'],
          ['-', '+', '-', '+'],
          ['-', '-', '+', '+'],
          ['-', '+', '+', '+'],
          ]
)

In [96]:
df_tablica_pokryc

,-0-0,-1-1,--1-,1---
0,+,-,-,-
2,+,-,+,-
3,-,-,+,-
5,-,+,-,-
6,-,-,+,-
7,-,+,+,-
8,+,-,-,+
9,-,-,-,+
10,+,-,+,+
11,-,-,+,+


In [99]:
assert create_tablica_pokryc(df_groupwane).equals(df_tablica_pokryc)

NameError: name 'create_tablica_pokryc' is not defined

In [ ]:
pd.DataFrame.()